In [4]:
import sys
import os

# 現在: mtsb/notebooks
# 1つ上は: mtsb/
PROJECT_ROOT = os.path.abspath("..")
sys.path.append(PROJECT_ROOT)

print("Added to sys.path:", PROJECT_ROOT)


Added to sys.path: c:\Users\AVRJ0006\YamatoDX\YAMATO_learning_practice\my_time_series_benchmark\project_git\My-Time-Series-Benchmark\mtsb


In [ ]:
# =========================================================
#  インポート（パス構造・モジュール名すべて naive.ipynb と同じ）
# =========================================================
import pandas as pd
import numpy as np
from pathlib import Path

# モデル（※ models/baseline/seasonal_naive.py）
from models.baseline.seasonal_naive import SeasonalNaiveModel

# 指標（naive.ipynb と同じフォルダ階層）
from evaluation.basic.mae import MAE
from evaluation.basic.rmse import RMSE
from evaluation.basic.mape import MAPE
from evaluation.basic.smape import SMAPE
from evaluation.basic.r2 import R2


In [ ]:
# =========================================================
#  データ読み込み（パスは naive.ipynb と完全一致）
# =========================================================

# mtsb/notebooks から見て1つ上 → ../data/raw/daily_sales.csv
df = pd.read_csv("../data/raw/daily_sales.csv")
df["date"] = pd.to_datetime(df["date"])
df = df.set_index("date")

y = df["sales"]

# Naive.ipynb と同じ horizon
horizon = 30

# train/test 分割（DatasetLoader を使わない点も naive と同じ）
train = y.iloc[:-horizon]
test = y.iloc[-horizon:]

train.head(), test.head()


(date
 2023-01-01    127.94
 2023-01-02    133.14
 2023-01-03    129.73
 2023-01-04    122.88
 2023-01-05    124.41
 Name: sales, dtype: float64,
 date
 2023-06-20    133.05
 2023-06-21    134.51
 2023-06-22    125.49
 2023-06-23    130.29
 2023-06-24    133.10
 Name: sales, dtype: float64)

In [ ]:
# =========================================================
#  Seasonal Naive モデルの学習 & 予測
# =========================================================

# 日次データ → 週周期の場合
seasonal_period = 7

model = SeasonalNaiveModel(freq="D", seasonal_period=seasonal_period)
model.fit(train)

y_pred = model.forecast(horizon)

y_pred.head()


2023-06-20    146.65
2023-06-21    147.65
2023-06-22    126.62
2023-06-23    130.89
2023-06-24    122.83
Freq: D, Name: seasonal_naive, dtype: float64

In [ ]:
# =========================================================
#  評価指標（Naive.ipynb と全く同じ構造）
# =========================================================
metrics = {
    "mae": MAE().compute(test, y_pred),
    "rmse": RMSE().compute(test, y_pred),
    "mape": MAPE().compute(test, y_pred),
    "smape": SMAPE().compute(test, y_pred),
    "r2": R2().compute(test, y_pred),
}

metrics


{'mae': 18.862000000000002,
 'rmse': 22.119055585625713,
 'mape': 12.90858796446564,
 'smape': 14.044218684057604,
 'r2': -2.7069058684161447}

In [ ]:
# =========================================================
#  horizon 別誤差（naive.ipynb と同一ロジック）
# =========================================================
horizon_rows = []
for h in range(1, horizon + 1):
    yt = test.iloc[h-1]
    yp = y_pred.iloc[h-1]

    horizon_rows.append({
        "horizon": h,
        "abs_error": abs(yt - yp),
        "squared_error": (yt - yp) ** 2,
        "pct_error": abs(yt - yp) / abs(yt) * 100 if yt != 0 else None,
    })

horizon_df = pd.DataFrame(horizon_rows)
horizon_df.head()


,horizon,abs_error,squared_error,pct_error
0,1,13.60,184.9600,10.221721
1,2,13.14,172.6596,9.768790
2,3,1.13,1.2769,0.900470
3,4,0.60,0.3600,0.460511
4,5,10.27,105.4729,7.716003


In [ ]:
# =========================================================
#  残差分析（naive.ipynb と完全一致）
# =========================================================
residual = test - y_pred

residual_df = pd.DataFrame({
    "residual": residual,
    "acf_1": residual.autocorr(1),
    "acf_7": residual.autocorr(7),
    "acf_14": residual.autocorr(14),
})

residual_df.head()


,residual,acf_1,acf_7,acf_14
date,,,,
2023-06-20,-13.60,0.805995,0.544899,-0.120748
2023-06-21,-13.14,0.805995,0.544899,-0.120748
2023-06-22,-1.13,0.805995,0.544899,-0.120748
2023-06-23,-0.60,0.805995,0.544899,-0.120748
2023-06-24,10.27,0.805995,0.544899,-0.120748


In [ ]:
# =========================================================
#  CSV 出力（パスもファイル名も naive.ipynb と同じ構造）
# =========================================================

# forecast
forecast_df = pd.DataFrame({
    "date": y_pred.index,
    "actual": test.values,
    "forecast": y_pred.values,
    "model": "seasonal_naive",
})
forecast_df.to_csv("../../exports_for_tableau/seasonal_naive_forecast.csv", index=False)

# metrics
metrics_df = pd.DataFrame([{
    "model": "seasonal_naive",
    **metrics
}])
metrics_df.to_csv("../../exports_for_tableau/seasonal_naive_metrics.csv", index=False)

# horizon error
horizon_df.to_csv("../../e" \
"xports_for_tableau/seasonal_naive_horizon_error.csv", index=False)

# residual
pd.DataFrame({
    "date": residual.index,
    "residual": residual.values
}).to_csv("../../exports_for_tableau/seasonal_naive_residual.csv", index=False)

print("CSV files exported successfully!")


CSV files exported successfully!
